In [5]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
model="llama3"
llm = ChatOllama(model=model)
# prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# /docs/concepts/#langchain-expression-language-lcel
# chain = prompt | llm | StrOutputParser()

# topic = {"topic": "Space travel"}

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
# print(chain.invoke({"topic": "Space travel"}))
# async for chunks in chain.astream(topic):
#     print(chunks, end="", flush=True)

Why did the astronaut break up with his girlfriend before going to Mars?

Because he needed space!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

model="llama3"
llm = ChatOllama(model=model)
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(llm, get_session_history)

config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run afc4e6ef-cef5-4e67-ab24-ce0a3c2152f6 not found for run 5ff2cb74-e329-4739-ab14-722c5552d418. Treating as a root run.


"Hi Bob! It's nice to meet you. Is there something on your mind that you'd like to chat about, or are you just looking for some friendly conversation?"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 1cb9abf3-5bb0-4c72-ad59-4eb496a8ab71 not found for run 9992ef4a-1ea0-4a39-a798-b1d2e8f0d4c0. Treating as a root run.


'Your name is Bob!'

In [13]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 7fe6965d-91f9-4ebe-b77c-6be6ded9a548 not found for run efa050c9-cfd6-4cb8-97fc-ab6bf2d3d4d6. Treating as a root run.


"I'm afraid I don't know your name! As a conversational AI, I don't have any prior knowledge about you or your personal information. Each time you interact with me, it's a new conversation and we start from scratch. Would you like to introduce yourself, though?"

In [15]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="my age is 25 and i like to swim and write code")],
    config=config,
)

response.content

Parent run efca858a-fc7a-4929-9163-437e2868ef79 not found for run a58e4bd1-ddab-4c5e-b8fb-25d9bb9cbe16. Treating as a root run.


"So, you're a young and tech-savvy individual who enjoys swimming and coding. That's awesome!\n\nAt 25, you're likely in the midst of exploring your passions and interests, and it sounds like you've found two activities that bring you joy. Swimming is an excellent way to stay physically fit and mentally relaxed, while writing code can be a fun and challenging creative outlet.\n\nWhat kind of coding do you enjoy doing? Are you working on any personal projects or contributing to open-source initiatives?"

In [16]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="what is my name and age")],
    config=config,
)

response.content

Parent run 7849c14b-cd6c-4877-a724-8269d7cf151c not found for run c1959486-5de5-4eb8-9794-431461128725. Treating as a root run.


'According to our conversation earlier, your name is Bob and your age is 25!'

In [17]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="what are my interests")],
    config=config,
)

response.content

Parent run ef3185a1-9aa9-4a4f-b97a-cd4fb62918f7 not found for run b4698a9b-d078-4c5c-9a42-225e91bc3ef1. Treating as a root run.


'I remember! According to our chat, your interests include swimming and writing code!'